This notebook implements a complete **Agentic RAG system using LangGraph** for the **Supply Chain** domain.

In [ ]:

from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool

from dotenv import load_dotenv
from typing import Literal
import os


In [ ]:

# Load environment variables
load_dotenv()

API_KEY = os.getenv("paid_api")
if not API_KEY:
    raise ValueError("API key not found in .env")

llm = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0.4,
    api_key=API_KEY
)

print("LLM initialized")


In [ ]:

# Load Supply Chain documents (PDFs)
DOC_PATH = r"C:\Users\owner\Desktop\Agentic_RAG_SupplyChain_KB"

loader = PyPDFDirectoryLoader(DOC_PATH)
documents = loader.load()

print(f"Loaded {len(documents)} documents")


In [ ]:

# Chunk documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=75
)

doc_chunks = text_splitter.split_documents(documents)
print(f"Created {len(doc_chunks)} chunks")


In [ ]:

# Create vector store
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=API_KEY
)

VECTOR_DB_PATH = "./chroma_supplychain_rag"

vectorstore = Chroma(
    collection_name="supplychain_docs",
    embedding_function=embeddings,
    persist_directory=VECTOR_DB_PATH
)

vectorstore.add_documents(doc_chunks)
print("Vector store ready")


In [ ]:

@tool
def retrieve_supplychain_docs(query: str) -> str:
    """Retrieve supply chain documents for domain-specific questions."""
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 12}
    )
    docs = retriever.invoke(query)

    if not docs:
        return "No relevant documents found."

    return "\n\n---\n\n".join(
        f"Source {i+1}:\n{doc.page_content}"
        for i, doc in enumerate(docs)
    )


In [ ]:

system_prompt = SystemMessage(content="""
You are SupplyChainBot, an expert assistant in Supply Chain Management.

Retrieve documents only when supply-chain-specific knowledge is required.
Do NOT retrieve for greetings, math, dates, or general knowledge.
Cite retrieved information clearly.
""")


In [ ]:

tools = [retrieve_supplychain_docs]
llm_with_tools = llm.bind_tools(tools)


In [ ]:

def assistant(state: MessagesState) -> dict:
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"


In [ ]:

builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
agent = builder.compile(checkpointer=memory)

print("Agentic RAG compiled")


In [ ]:

def query_agent(queries, thread_id="test"):
    for q in queries:
        print("=" * 70)
        print("Query:", q)
        print("=" * 70)

        result = agent.invoke(
            {"messages": [HumanMessage(content=q)]},
            config={"configurable": {"thread_id": thread_id}}
        )

        used_retrieval = any(
            isinstance(m, AIMessage) and m.tool_calls
            for m in result["messages"]
        )

        print("Answer:", result["messages"][-1].content)
        print("Decision:", "RETRIEVED" if used_retrieval else "ANSWERED DIRECTLY")
        print()
